In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque
import traci
import sumolib
import time

environment = "environments/cross.sumocfg"
sumobin = sumolib.checkBinary('sumo-gui')

traci.start([sumobin, '-c', environment])  


print("Connected to TraCI")

Connected to TraCI


In [2]:
# environment = "./environments/cross.sumocfg"

# Function to reset the SUMO environment
def reset_sumo_environment(config_file):
    # conn = traci.getConnection('default')
    # Get initial state information (modify this based on your state representation)
    state = get_state()

    print(state)
    
    return state

# Function to step through the SUMO simulation
def step_in_sumo(action, tls_id="0"):
    # Apply the action
    apply_action(action)
    
    # Step the SUMO simulation forward
    conn = traci.getConnection('default')
    conn.simulationStep()
    
    # Get the new state after taking the action
    next_state = get_state()
    
    # Calculate the reward with the specified tls_id
    reward = calculate_reward(tls_id)
    
    # Check if the episode is done
    done = check_done_condition()
    
    return next_state, reward, done

# Function to get the current state (modify this based on what information you need)
def get_state():
    # Example: returning the number of vehicles on the road
    traffic_light_ids = traci.trafficlight.getIDList()
    current_state = []
    
    for tl_id in traffic_light_ids:
    # Add more features as needed (e.g., speeds, positions, etc.)
        lights_state = traci.trafficlight.getRedYellowGreenState(tl_id)
        # Mapping dictionary
        char_to_int = {'r': 0, 'y': 1, 'G': 2}
        lights_state = [char_to_int[char] for char in lights_state]
        current_state.append(lights_state)

    print("CURRENT STATE:", current_state)
    return current_state 

# Function to apply the action (modify based on your action space)
def apply_action(action):
    traffic_light_ids = traci.trafficlight.getIDList()
    for tl_id in traffic_light_ids:
        try:
            current_state = traci.trafficlight.getRedYellowGreenState(tl_id)
            phase_length = len(current_state)
            print(f"Traffic light {tl_id} current state: {current_state}, length: {phase_length}")
            
            if action == 0:
                # Set all lanes to red
                new_state = "r" * phase_length
                print(f"Setting {tl_id} to {new_state}")
                traci.trafficlight.setRedYellowGreenState(tl_id, new_state)
            elif action == 1:
                # Set all lanes to green
                new_state = "y" * phase_length
                print(f"Setting {tl_id} to {new_state}")
                traci.trafficlight.setRedYellowGreenState(tl_id, new_state)
            elif action == 2:
                # Set all lanes to green
                new_state = "G" * phase_length
                print(f"Setting {tl_id} to {new_state}")
                traci.trafficlight.setRedYellowGreenState(tl_id, new_state)
        except traci.TraCIException as e:
            print(f"Error setting traffic light {tl_id}: {e}")

# Function to calculate the reward (implement your logic)
def calculate_reward(tls_id):
    # Retrieve delays, waiting times, and emergency stops for the given traffic light
    delay = get_delay(tls_id)
    waiting_time = get_waiting_time(tls_id)
    emergency_stops = num_emergency_stops(tls_id)
    
    # Weights for each component in the reward function (adjust as needed)
    delay_weight = -1.0      # Negative reward for higher delays
    waiting_time_weight = -0.5  # Negative reward for higher waiting times
    emergency_stop_weight = -2.0  # Strong negative reward for emergency stops

    # Compute the reward as a weighted sum of the three components
    reward = (delay_weight * delay) + (waiting_time_weight * waiting_time) + (emergency_stop_weight * emergency_stops)
    
    # Optionally, normalize reward if values are large or small
    # reward = reward / (1 + abs(reward))
    
    print(f"Reward calculated: Delay={delay}, Waiting Time={waiting_time}, Emergency Stops={emergency_stops}, Total Reward={reward}")
    
    return reward

# Function to sum the delay of all vehicles affected by the traffic light
def get_delay(tls_id):
    delays = [get_lane_delay(lane_id) for lane_id in traci.trafficlight.getControlledLanes(tls_id)]
    return sum(delays)

def get_lane_delay(lane_id):
    max_s = traci.lane.getMaxSpeed(lane_id)
    avg_s = traci.lane.getLastStepMeanSpeed(lane_id)
    num_veh = traci.lane.getLastStepVehicleNumber(lane_id)
    return num_veh * (1 - avg_s / max_s)

# Function to get the sum of waiting times of all vehicles currently stopped at the traffic light
def get_waiting_time(tls_id):
    waiting_times = [get_lane_waiting_time(lane_id) for lane_id in traci.trafficlight.getControlledLanes(tls_id)]
    return sum(waiting_times)

def get_lane_waiting_time(lane_id):
    waiting_times = [traci.vehicle.getWaitingTime(veh_id) for veh_id in traci.lane.getLastStepVehicleIDs(lane_id)]
    return sum(waiting_times)

# Function that returns the number of emergency stops (acceleration < -4.5m/s^2) caused by the traffic light
def num_emergency_stops(tls_id):
    emergency_stops = [get_lane_emergency_stops(lane_id) for lane_id in traci.trafficlight.getControlledLanes(tls_id)]
    return sum(emergency_stops)

def get_lane_emergency_stops(lane_id):
    emergency_stops = [veh_id for veh_id in traci.lane.getLastStepVehicleIDs(lane_id) if traci.vehicle.getAcceleration(veh_id) < -4.5]
    return len(emergency_stops)

# Function to check if the simulation should terminate
def check_done_condition():
    # Example condition: terminate if simulation time exceeds a limit
    current_time = traci.simulation.getTime()
    return current_time > 1000  # Change this threshold as necessary

In [3]:
# Define the neural network for the Q-function
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 24)
        self.fc2 = nn.Linear(24, 24)
        self.fc3 = nn.Linear(24, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [4]:
# Define the RL agent
class RLAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 0.6  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = DQN(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.criterion = nn.MSELoss()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size + 1)
        state = torch.FloatTensor(state)
        q_values = self.model(state)
        return np.argmax(q_values.detach().numpy())

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.amax(self.model(torch.FloatTensor(next_state)).detach().numpy())
            target_f = self.model(torch.FloatTensor(state))
            # Check if action index is valid
            if 0 <= action < self.action_size:
                target_f[action] = target
            else:
                print(f"Invalid action: {action}")

            # Convert back to tensor for loss calculation
            target_f_tensor = torch.FloatTensor(target_f)
            self.model.zero_grad()
            loss = self.criterion(target_f_tensor, self.model(torch.FloatTensor(state)))
            loss.backward()
            self.optimizer.step()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [5]:
# Simulation interaction loop
def run_simulation(agent, num_episodes, batch_size):
    for e in range(num_episodes):
        state = reset_sumo_environment(environment)  # Reset the SUMO environment and get the initial state
        done = False
        total_reward = 0

        while not done:
            action = agent.act(state)
            next_state, reward, done = step_in_sumo(action)  # Step through the SUMO simulation
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward

        print(f"Episode: {e+1}/{num_episodes}, Total Reward: {total_reward}")
        agent.replay(batch_size)

In [6]:
# Sample parameters
action_size = traci.trafficlight.getIDCount()  # Example action size, adjust based on your simulation
state_size = 0
traffic_light_ids = traci.trafficlight.getIDList()

for tl_id in traffic_light_ids:
    lights_state = traci.trafficlight.getRedYellowGreenState(tl_id)
    phase_length = len(lights_state)
    state_size += phase_length
print("action_size:", action_size)
agent = RLAgent(state_size, action_size)
run_simulation(agent, num_episodes=1000, batch_size=32)

action_size: 1
CURRENT STATE: [[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]]
[[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]]
Traffic light 0 current state: rryrrrrryrrr, length: 12
Setting 0 to rrrrrrrrrrrr
CURRENT STATE: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Reward calculated: Delay=0.0, Waiting Time=0, Emergency Stops=0, Total Reward=-0.0
Traffic light 0 current state: rrrrrrrrrrrr, length: 12
Setting 0 to rrrrrrrrrrrr
CURRENT STATE: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Reward calculated: Delay=0.0, Waiting Time=0, Emergency Stops=0, Total Reward=-0.0
Traffic light 0 current state: rrrrrrrrrrrr, length: 12
Setting 0 to rrrrrrrrrrrr
CURRENT STATE: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Reward calculated: Delay=0.0, Waiting Time=0, Emergency Stops=0, Total Reward=-0.0
Traffic light 0 current state: rrrrrrrrrrrr, length: 12
Setting 0 to rrrrrrrrrrrr
CURRENT STATE: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Reward calculated: Delay=0.0, Waiting Time=0, Emergency Stops=0, Total Reward=-0.0
Traffic light 0 